In [22]:
#notebook for scrape all necessary player stats nad lineups from web
#import necessary packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import csv
import time

In [27]:
#method for scraping links for all games of a EPL season
def get_links(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    test = soup.find(id='div_sched_2021-2022_9_1')
    x = "score"
    test = test.findAll("td", {"data-stat": x})
    link_list = []
    for i in test:
        i = i.find("a")
        if i is not None:
            link = i['href']
            link_list.append(link)
            #print(link)
    return link_list

In [28]:
#@title

#stats matchday
statsm = ["player", "shirtnumber", "nationality", "position", "age", "minutes", "goals", "assists", "touches", "sca",
          "gca", 'pens_made', 'shots_total', 'shots_on_target', 'pressures', 'tackles', 'interceptions', 'blocks', 'xg',
          'xa', 'passes', 'passes_pct', 'progressive_passes', 'progressive_carries', 'dribbles_completed', 'dribbles']

#stats keeper matchday
statskeepers = ['player', 'minutes', 'age', 'gk_shots_on_target_against', 'gk_goals_against', 'gk_saves', 'gk_save_pct',
                'gk_psxg', 'gk_passes_completed_launched', 'gk_passes', 'gk_passes_throws', 'gk_pct_passes_launched',
                'gk_def_actions_outside_pen_area']

keepers = ["player", "nationality", "position", "squad", "age", "birth_year", "games_gk", "games_starts_gk",
           "minutes_gk", "goals_against_gk", "goals_against_per90_gk", "shots_on_target_against", "saves", "save_pct",
           "wins_gk", "draws_gk", "losses_gk", "clean_sheets", "clean_sheets_pct", "pens_att_gk", "pens_allowed",
           "pens_saved", "pens_missed_gk"]
keepers3 = ["players_used", "games_gk", "games_starts_gk", "minutes_gk", "goals_against_gk", "goals_against_per90_gk",
            "shots_on_target_against", "saves", "save_pct", "wins_gk", "draws_gk", "losses_gk", "clean_sheets",
            "clean_sheets_pct", "pens_att_gk", "pens_allowed", "pens_saved", "pens_missed_gk"]


#advance goalkeeping(keepersadv)


#Functions to get the data in a dataframe using BeautifulSoup

def get_tables_m(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    matchweek = soup.find(id='content').find("div").contents[1]
    date = soup.find("div", {"class": "scorebox_meta"}).find("div").contents[0].contents[0]
    date = date['href']
    date = date[12:22]
    #date = soup.find("div", {"class": "scorebox_meta"}).soup.find("span", {"class": "venuetime"})

    team_H = soup.findAll("strong")[2].find("a").contents[0]
    if soup.findAll("strong")[5].find("a") is None:
        team_A = soup.findAll("strong")[4].find("a").contents[0]
    else:
        team_A = soup.findAll("strong")[5].find("a").contents[0]

    team1_table = all_tables[0]
    team2_table = all_tables[7]
    gk1_table = all_tables[6]
    gk2_table = all_tables[13]

    #gk1_table.append(gk2_table)
    #team1_table.append(team2_table)

    return team1_table, team2_table, gk1_table, gk2_table, matchweek, date, team_H, team_A


def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    rows_player
    #print(rows_player)
    for row in rows_player:
        #print(row)
        if (row.find('th', {"scope": "row"}) != None):

            for f in features_wanted_player:
                #print(f)
                cell = row.find("td", {"data-stat": f})
                #print(cell)
                if cell is None:
                    text = row.find('th', {"scope": "row"}).find('a').contents[0]
                    if f in pre_df_player:
                        pre_df_player[f].append(text)
                    else:
                        pre_df_player[f] = [text]
                #print(cell)
                else:
                    a = cell.text.strip().encode()
                    text = a.decode("utf-8")
                    #print(text)
                    if (text == ''):
                        text = '0'
                    if ((f != 'player') & (f != 'nationality') & (f != 'position') & (f != 'squad') & (f != 'age') & (
                            f != 'birth_year')):
                        text = float(text.replace(',', ''))
                    if f in pre_df_player:
                        pre_df_player[f].append(text)
                    else:
                        pre_df_player[f] = [text]
    #print(pre_df_player)
    df_player = pd.DataFrame.from_dict(pre_df_player)
    #print(df_player)
    return df_player


def frame_for_category(category, top, end, features):
    url = (top + category + end)
    print(url)
    player_table_H, player_table_A, gk_table_H, gk_table_A, matchweek, date, team_H, team_A = get_tables_m(url)
    #print(player_table)
    df_player_H = get_frame(features, player_table_H)
    df_player_H['Team'] = team_H
    df_player_A = get_frame(features, player_table_A)
    df_player_A['Team'] = team_A
    df_player = pd.concat([df_player_H, df_player_A])
    df_player['matchweek'] = matchweek
    df_player['date'] = date
    df_gk_H = get_frame(statskeepers, gk_table_H)
    df_gk_H['Team'] = team_H
    df_gk_A = get_frame(statskeepers, gk_table_A)
    df_gk_A['Team'] = team_A
    df_gk = pd.concat([df_gk_H, df_gk_A])
    df_gk['matchweek'] = matchweek
    df_gk['date'] = date
    return df_player, df_gk


def get_matchday_data(top, end, linklist):
    pl, gk = frame_for_category('/en/matches/3adf2aa7', top, end, statsm)
    #pl1, gk1 = frame_for_category('/en/matches/3adf2aa7', top, end, statsm)

    for i in linklist:
        tmppl, tmpgk = frame_for_category(i, top, end, statsm)
        pl = pd.concat([pl, tmppl], axis=0)
        gk = pd.concat([gk, tmpgk], axis=0)
        time.sleep(5)

    return pl, gk


#pl, gk = frame_for_category('/en/matches/51fb894e', 'https://fbref.com/', '', statsm)
#gk

In [29]:
def get_lineups(url):
    pre_df_player = dict()
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    date = soup.find("div", {"class": "scorebox_meta"}).find("div").contents[0].contents[0]
    date = date['href']
    date = date[12:22]
    matchweek = soup.find(id='content').find("div").contents[1]
    lineup = soup.find_all("div", {"class": "lineup"})
    lineup_H = lineup[0]
    lineup_A = lineup[1]

    rows_lineup_H = lineup_H.find_all('tr')
    rows_lineup_A = lineup_A.find_all('tr')
    lineup_total = [rows_lineup_H, rows_lineup_A]
    #print(rows_lineup)
    i = 0


    for rows_lineup in lineup_total:
        i = i+1
        for row in rows_lineup:

            if row.find('a') != None:
                #print(row.find('a').contents[0])
                cell = row.find('a').contents[0]
                a = cell.text.strip().encode()
                text = a.decode("utf-8")

                if 'player' in pre_df_player:
                    pre_df_player['player'].append(text)
                    #pre_df_player['count'].append(3)
                else:
                    pre_df_player['player'] = [text]
                    #pre_df_player['count'] = i

        if i == 1:
            df_player_H = pd.DataFrame.from_dict(pre_df_player)
        else: df_player_A = pd.DataFrame.from_dict(pre_df_player)
        pre_df_player = dict()



    team_H = soup.findAll("strong")[2].find("a").contents[0]
    if soup.findAll("strong")[5].find("a") is None:
        team_A = soup.findAll("strong")[4].find("a").contents[0]
    else:
        team_A = soup.findAll("strong")[5].find("a").contents[0]

    #gk1_table.append(gk2_table)
    #team1_table.append(team2_table)

    return df_player_H, df_player_A, team_H, team_A, matchweek, date

def frame_for_category_lineups (category, top, end):
    url = (top + category + end)
    print(url)
    player_H, player_A, teamH, teamA, matchweek, date,= get_lineups(url)

    player_H['team'] = teamH
    player_A['team'] = teamA
    df_player = pd.concat([player_H, player_A])
    df_player['matchweek'] = matchweek
    df_player['date'] = date
    df_player['start11'] = np.where(df_player.index< 11, True, False)

    return df_player

def get_matchday_data_lineups(top, end, linklist):
    pl = frame_for_category_lineups('/en/matches/3adf2aa7', top, end)
    #pl1, gk1 = frame_for_category('/en/matches/3adf2aa7', top, end, statsm)

    for i in linklist:
        tmppl = frame_for_category_lineups(i, top, end)
        pl = pd.concat([pl, tmppl], axis=0)
        time.sleep(5)

    return pl


#H, A, teamH, teamA, matchweek, date = get_lineups(
 #   'https://fbref.com/en/matches/4a652b81/Tottenham-Hotspur-West-Bromwich-Albion-February-7-2021#-Premier-League')
#frame_for_category_lineups('/en/matches/51fb894e', 'https://fbref.com/', '')

In [36]:
link_list = get_links('https://fbref.com/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures')

# split list in 4 parts to not bring down website
list1 = []
list2 = []
list3 = []
list4 = []
count = 0

for i in link_list:
    if count < 100:
        list1.append(i)
    elif count < 200:
        list2.append(i)
    elif count < 300:
        list3.append(i)
    else:
        list4.append(i)
    count = count + 1

In [43]:
#scrape data and store it in df's for players and goalkeepers
df_1, gk_1 = get_matchday_data('https://fbref.com/', '', list1)
df_2, gk_2 = get_matchday_data('https://fbref.com/', '', list2)
df_3, gk_3 = get_matchday_data('https://fbref.com/', '', list3)
df_4, gk_4 = get_matchday_data('https://fbref.com/', '', list4)

#df_test, gk_test = get_matchday_data('https://fbref.com/', '', link_list )

https://fbref.com//en/matches/3adf2aa7
https://fbref.com//en/matches/f976e20a/Everton-Manchester-United-April-9-2022-Premier-League
https://fbref.com//en/matches/37ddcda6/Arsenal-Brighton-and-Hove-Albion-April-9-2022-Premier-League
https://fbref.com//en/matches/70d15cf0/Southampton-Chelsea-April-9-2022-Premier-League
https://fbref.com//en/matches/b18deecf/Watford-Leeds-United-April-9-2022-Premier-League
https://fbref.com//en/matches/811d471f/Aston-Villa-Tottenham-Hotspur-April-9-2022-Premier-League
https://fbref.com//en/matches/8574aaf3/Norwich-City-Burnley-April-10-2022-Premier-League
https://fbref.com//en/matches/6bb36736/Brentford-West-Ham-United-April-10-2022-Premier-League
https://fbref.com//en/matches/816c435d/Leicester-City-Crystal-Palace-April-10-2022-Premier-League
https://fbref.com//en/matches/37e2fe92/Manchester-City-Liverpool-April-10-2022-Premier-League
https://fbref.com//en/matches/0b266073/Tottenham-Hotspur-Brighton-and-Hove-Albion-April-16-2022-Premier-League
https://fb

In [44]:
df_4

,player,shirtnumber,nationality,position,age,minutes,goals,assists,touches,sca,...,xa,passes,passes_pct,progressive_passes,progressive_carries,dribbles_completed,dribbles,Team,matchweek,date
0,Ivan Toney,17.0,eng ENG,FW,25-150,90.0,0.0,0.0,29.0,1.0,...,0.1,21.0,47.6,3.0,0.0,1.0,2.0,Brentford,(Matchweek 1),2021-08-13
1,Bryan Mbeumo,19.0,cm CMR,FW,22-006,85.0,0.0,0.0,34.0,3.0,...,0.1,23.0,65.2,1.0,1.0,2.0,3.0,Brentford,(Matchweek 1),2021-08-13
2,Marcus Forss,9.0,fi FIN,FW,22-056,5.0,0.0,0.0,5.0,0.0,...,0.0,3.0,66.7,0.0,1.0,1.0,2.0,Brentford,(Matchweek 1),2021-08-13
3,Vitaly Janelt,27.0,de GER,CM,23-095,90.0,0.0,0.0,41.0,1.0,...,0.0,29.0,75.9,4.0,2.0,0.0,0.0,Brentford,(Matchweek 1),2021-08-13
4,Frank Onyeka,15.0,ng NGA,CM,23-224,79.0,0.0,0.0,22.0,2.0,...,0.1,14.0,92.9,3.0,0.0,1.0,1.0,Brentford,(Matchweek 1),2021-08-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,Adam Masina,11.0,ma MAR,LB,28-140,13.0,0.0,1.0,6.0,1.0,...,0.2,5.0,100.0,0.0,0.0,0.0,0.0,Watford,(Matchweek 38),2022-05-22
10,Samir Santos,22.0,br BRA,CB,27-168,90.0,0.0,0.0,30.0,0.0,...,0.0,24.0,62.5,0.0,1.0,0.0,0.0,Watford,(Matchweek 38),2022-05-22
11,Christian Kabasele,27.0,be BEL,CB,31-087,90.0,0.0,0.0,21.0,0.0,...,0.0,14.0,71.4,0.0,0.0,0.0,0.0,Watford,(Matchweek 38),2022-05-22
12,Kiko Femenía,21.0,es ESP,RB,31-109,90.0,0.0,0.0,58.0,0.0,...,0.0,47.0,63.8,4.0,4.0,0.0,2.0,Watford,(Matchweek 38),2022-05-22


In [52]:
#scrape lineups
l_1 = get_matchday_data_lineups('https://fbref.com/', '', list1)
l_2 = get_matchday_data_lineups('https://fbref.com/', '', list2)
l_3 = get_matchday_data_lineups('https://fbref.com/', '', list3)
l_4 = get_matchday_data_lineups('https://fbref.com/', '', list4)

https://fbref.com//en/matches/3adf2aa7
https://fbref.com//en/matches/3adf2aa7/Brentford-Arsenal-August-13-2021-Premier-League
https://fbref.com//en/matches/e62685d4/Manchester-United-Leeds-United-August-14-2021-Premier-League
https://fbref.com//en/matches/814b563c/Watford-Aston-Villa-August-14-2021-Premier-League
https://fbref.com//en/matches/6f454493/Chelsea-Crystal-Palace-August-14-2021-Premier-League
https://fbref.com//en/matches/c99ebbf5/Everton-Southampton-August-14-2021-Premier-League
https://fbref.com//en/matches/4eb36e37/Burnley-Brighton-and-Hove-Albion-August-14-2021-Premier-League
https://fbref.com//en/matches/0b346a62/Leicester-City-Wolverhampton-Wanderers-August-14-2021-Premier-League
https://fbref.com//en/matches/c52500ad/Norwich-City-Liverpool-August-14-2021-Premier-League
https://fbref.com//en/matches/41091264/Newcastle-United-West-Ham-United-August-15-2021-Premier-League
https://fbref.com//en/matches/ff51efc7/Tottenham-Hotspur-Manchester-City-August-15-2021-Premier-Leag

In [ ]:
l_1

In [45]:
def concatenate_files(f1, f2, f3, f4):
    total = pd.concat([f1, f2, f3, f4], axis=0)
    total = total.drop_duplicates()
    total.reset_index(inplace=True, drop=True)
    return total

In [46]:
gk = concatenate_files(gk_1, gk_2, gk_3, gk_4)
pl = concatenate_files(df_1, df_2, df_3, df_4)

In [53]:
lineups = concatenate_files(l_1, l_2, l_3, l_4)

In [47]:
pl['matchweek'] = pl['matchweek'].str.slice(start=-3, stop=-1).astype(float)
gk['matchweek'] = gk['matchweek'].str.slice(start=-3, stop=-1).astype(float)
pl['position'] = pl['position'].str.slice(start=0, stop=2)
gk

,player,minutes,age,gk_shots_on_target_against,gk_goals_against,gk_saves,gk_save_pct,gk_psxg,gk_passes_completed_launched,gk_passes,gk_passes_throws,gk_pct_passes_launched,gk_def_actions_outside_pen_area,Team,matchweek,date
0,David Raya,90.0,25-332,4.0,0.0,4.0,100.0,0.4,10.0,35.0,5.0,60.0,0.0,Brentford,1.0,2021-08-13
1,Bernd Leno,90.0,29-162,3.0,2.0,1.0,33.3,0.9,3.0,26.0,3.0,19.2,0.0,Arsenal,1.0,2021-08-13
2,David de Gea,90.0,30-280,3.0,1.0,2.0,66.7,0.6,9.0,23.0,3.0,17.4,0.0,Manchester United,1.0,2021-08-14
3,Illan Meslier,90.0,21-165,8.0,5.0,3.0,37.5,1.9,8.0,38.0,10.0,42.1,1.0,Leeds United,1.0,2021-08-14
4,Daniel Bachmann,90.0,27-036,1.0,2.0,0.0,0.0,1.5,7.0,24.0,5.0,50.0,0.0,Watford,1.0,2021-08-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,John Ruddy,45.0,35-210,5.0,2.0,3.0,60.0,1.4,2.0,7.0,1.0,85.7,0.0,Wolverhampton Wanderers,38.0,2022-05-22
759,Ederson,90.0,28-278,2.0,2.0,0.0,0.0,0.7,9.0,22.0,3.0,36.4,0.0,Manchester City,38.0,2022-05-22
760,Robin Olsen,90.0,32-134,5.0,3.0,2.0,40.0,2.6,11.0,28.0,6.0,64.3,0.0,Aston Villa,38.0,2022-05-22
761,Edouard Mendy,90.0,30-082,3.0,1.0,2.0,66.7,1.1,2.0,21.0,3.0,9.5,2.0,Chelsea,38.0,2022-05-22


In [ ]:
pl

In [54]:
lineups['matchweek'] = lineups['matchweek'].str.slice(start=-3, stop=-1).astype(float)

In [55]:
lineups

,player,team,matchweek,date,start11
0,David Raya,Brentford,1.0,2021-08-13,True
1,Rico Henry,Brentford,1.0,2021-08-13,True
2,Ethan Pinnock,Brentford,1.0,2021-08-13,True
3,Christian Nørgaard,Brentford,1.0,2021-08-13,True
4,Sergi Canós,Brentford,1.0,2021-08-13,True
...,...,...,...,...,...
15158,Craig Cathcart,Watford,38.0,2022-05-22,False
15159,Dan Gosling,Watford,38.0,2022-05-22,False
15160,Cucho,Watford,38.0,2022-05-22,False
15161,Francisco Sierralta,Watford,38.0,2022-05-22,False


In [48]:
gk['position'] = 'GK'

In [49]:
#map position to category
def get_category(df):
    df.loc[df['position'] == 'GK', 'category'] = 'Goalkeeper'
    df.loc[df['position'] == 'DF', 'category'] = 'Defender'
    df.loc[df['position'] == 'RB', 'category'] = 'Defender'
    df.loc[df['position'] == 'LB', 'category'] = 'Defender'
    df.loc[df['position'] == 'CB', 'category'] = 'Defender'
    df.loc[df['position'] == 'WB', 'category'] = 'Defender'
    df.loc[df['position'] == 'FB', 'category'] = 'Defender'
    df.loc[df['position'] == 'MF', 'category'] = 'Midfield'
    df.loc[df['position'] == 'DM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'CM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'AM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'LM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'RM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'WM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'FW', 'category'] = 'Forward'
    df.loc[df['position'] == 'RW', 'category'] = 'Forward'
    df.loc[df['position'] == 'LW', 'category'] = 'Forward'
    return df


pl = get_category(pl)

In [56]:
#export lineups
def create_csv(df):
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    df.to_csv('PL2021_lineups.csv', index=False)
    return df


create_csv(lineups)


,player,team,matchweek,date,start11
0,David Raya,Brentford,1.0,2021-08-13,True
1,Rico Henry,Brentford,1.0,2021-08-13,True
2,Ethan Pinnock,Brentford,1.0,2021-08-13,True
3,Christian Nørgaard,Brentford,1.0,2021-08-13,True
4,Sergi Canós,Brentford,1.0,2021-08-13,True
...,...,...,...,...,...
15158,Craig Cathcart,Watford,38.0,2022-05-22,False
15159,Dan Gosling,Watford,38.0,2022-05-22,False
15160,Cucho,Watford,38.0,2022-05-22,False
15161,Francisco Sierralta,Watford,38.0,2022-05-22,False


In [50]:
#export goalkeeper data
def create_csv(df):
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    df.to_csv('PL2021_keepers.csv', index=False)
    return df


create_csv(gk)

,player,minutes,age,gk_shots_on_target_against,gk_goals_against,gk_saves,gk_save_pct,gk_psxg,gk_passes_completed_launched,gk_passes,gk_passes_throws,gk_pct_passes_launched,gk_def_actions_outside_pen_area,Team,matchweek,date,position
0,David Raya,90.0,25-332,4.0,0.0,4.0,100.0,0.4,10.0,35.0,5.0,60.0,0.0,Brentford,1.0,2021-08-13,GK
1,Bernd Leno,90.0,29-162,3.0,2.0,1.0,33.3,0.9,3.0,26.0,3.0,19.2,0.0,Arsenal,1.0,2021-08-13,GK
2,David de Gea,90.0,30-280,3.0,1.0,2.0,66.7,0.6,9.0,23.0,3.0,17.4,0.0,Manchester United,1.0,2021-08-14,GK
3,Illan Meslier,90.0,21-165,8.0,5.0,3.0,37.5,1.9,8.0,38.0,10.0,42.1,1.0,Leeds United,1.0,2021-08-14,GK
4,Daniel Bachmann,90.0,27-036,1.0,2.0,0.0,0.0,1.5,7.0,24.0,5.0,50.0,0.0,Watford,1.0,2021-08-14,GK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,John Ruddy,45.0,35-210,5.0,2.0,3.0,60.0,1.4,2.0,7.0,1.0,85.7,0.0,Wolverhampton Wanderers,38.0,2022-05-22,GK
759,Ederson,90.0,28-278,2.0,2.0,0.0,0.0,0.7,9.0,22.0,3.0,36.4,0.0,Manchester City,38.0,2022-05-22,GK
760,Robin Olsen,90.0,32-134,5.0,3.0,2.0,40.0,2.6,11.0,28.0,6.0,64.3,0.0,Aston Villa,38.0,2022-05-22,GK
761,Edouard Mendy,90.0,30-082,3.0,1.0,2.0,66.7,1.1,2.0,21.0,3.0,9.5,2.0,Chelsea,38.0,2022-05-22,GK


In [51]:
#export player data
def create_csv(df):
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    df.to_csv('PL2021_players.csv', index=False)
    return df


create_csv(pl)

,player,shirtnumber,nationality,position,age,minutes,goals,assists,touches,sca,...,passes,passes_pct,progressive_passes,progressive_carries,dribbles_completed,dribbles,Team,matchweek,date,category
0,Ivan Toney,17.0,eng ENG,FW,25-150,90.0,0.0,0.0,29.0,1.0,...,21.0,47.6,3.0,0.0,1.0,2.0,Brentford,1.0,2021-08-13,Forward
1,Bryan Mbeumo,19.0,cm CMR,FW,22-006,85.0,0.0,0.0,34.0,3.0,...,23.0,65.2,1.0,1.0,2.0,3.0,Brentford,1.0,2021-08-13,Forward
2,Marcus Forss,9.0,fi FIN,FW,22-056,5.0,0.0,0.0,5.0,0.0,...,3.0,66.7,0.0,1.0,1.0,2.0,Brentford,1.0,2021-08-13,Forward
3,Vitaly Janelt,27.0,de GER,CM,23-095,90.0,0.0,0.0,41.0,1.0,...,29.0,75.9,4.0,2.0,0.0,0.0,Brentford,1.0,2021-08-13,Midfield
4,Frank Onyeka,15.0,ng NGA,CM,23-224,79.0,0.0,0.0,22.0,2.0,...,14.0,92.9,3.0,0.0,1.0,1.0,Brentford,1.0,2021-08-13,Midfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10481,Adam Masina,11.0,ma MAR,LB,28-140,13.0,0.0,1.0,6.0,1.0,...,5.0,100.0,0.0,0.0,0.0,0.0,Watford,38.0,2022-05-22,Defender
10482,Samir Santos,22.0,br BRA,CB,27-168,90.0,0.0,0.0,30.0,0.0,...,24.0,62.5,0.0,1.0,0.0,0.0,Watford,38.0,2022-05-22,Defender
10483,Christian Kabasele,27.0,be BEL,CB,31-087,90.0,0.0,0.0,21.0,0.0,...,14.0,71.4,0.0,0.0,0.0,0.0,Watford,38.0,2022-05-22,Defender
10484,Kiko Femenía,21.0,es ESP,RB,31-109,90.0,0.0,0.0,58.0,0.0,...,47.0,63.8,4.0,4.0,0.0,2.0,Watford,38.0,2022-05-22,Defender
